In [24]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [25]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [26]:
n_samples, n_features = X.shape # số dòng là số mẫu, số cột là số đặc trưng
print(n_samples,n_features)

569 30


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=1234)

In [28]:
#scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [29]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [30]:
# chuyển y_train từ dạng 1 chiều (1D) sang dạng 2 chiều (2D) có 1 cột và số dòng bằng số mẫu.
y_train = y_train.view(y_train.shape[0],1)

y_test = y_test.view(y_test.shape[0],1)

In [31]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features,1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted


In [32]:
learning_rate = 0.01
n_iters = 500

In [33]:
model = LogisticRegression(n_features)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [34]:
for epoch in range(n_iters):
    y_predicted = model(X_train)

    loss = criterion(y_predicted, y_train)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

    if (epoch+1)%10:
        print(f'epoch:{epoch}',f'loss:{loss.item()}')

epoch:0 loss:0.6443321704864502
epoch:1 loss:0.6277638673782349
epoch:2 loss:0.6121492385864258
epoch:3 loss:0.5974216461181641
epoch:4 loss:0.583518385887146
epoch:5 loss:0.5703802704811096
epoch:6 loss:0.5579525232315063
epoch:7 loss:0.5461838841438293
epoch:8 loss:0.5350271463394165
epoch:10 loss:0.514378011226654
epoch:11 loss:0.5048084259033203
epoch:12 loss:0.49569591879844666
epoch:13 loss:0.4870092570781708
epoch:14 loss:0.47871971130371094
epoch:15 loss:0.4708009958267212
epoch:16 loss:0.4632289707660675
epoch:17 loss:0.4559813439846039
epoch:18 loss:0.44903767108917236
epoch:20 loss:0.4359881281852722
epoch:21 loss:0.42984896898269653
epoch:22 loss:0.4239465594291687
epoch:23 loss:0.4182674288749695
epoch:24 loss:0.4127986431121826
epoch:25 loss:0.4075285494327545
epoch:26 loss:0.40244626998901367
epoch:27 loss:0.3975416123867035
epoch:28 loss:0.39280518889427185
epoch:30 loss:0.383802205324173
epoch:31 loss:0.37951990962028503
epoch:32 loss:0.3753740191459656
epoch:33 loss:0

In [35]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc}')

accuracy: 0.9035087823867798
